In [4]:
import pandas as pd
import numpy as np
from scipy import spatial

TODO
* Para cada grupo X encontrar el grupo Y más cercano de la clasificación superior
* Seleccionar un grupo y comparar sus componentes con el grupo más cercano de la clasificación superior
* Ordenar el resultado de la comparación, de una manera util para el reporte

## Load data

In [5]:
df = pd.read_csv('dataset.csv')
df.head()

,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,TM,TG,PID,PF,PERS,AP,APO,knowledgeArea,classification,code
0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,11.0,6.0,...,0.00,0.91,0.67,3.36,1.82,0.00,0.0,Ingeniería y Tecnología,C,COL0033023
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3.0,...,0.00,0.00,1.09,1.82,1.82,0.00,0.0,Ciencias Naturales,C,COL0040519
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,0.00,1.68,0.00,0.00,3.36,3.28,0.0,Ciencias Médicas y de la Salud,A,COL0120245
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.36,0.00,0.36,0.00,0.00,1.82,0.0,Ingeniería y Tecnología,A,COL0110982
4,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,3.0,...,0.00,0.00,3.82,8.85,4.76,0.00,0.0,Ingeniería y Tecnología,B,COL0165935


In [8]:
# Knowledge areas
print(df['knowledgeArea'].unique())
# Classifications
print(df['classification'].unique())

['Ingeniería y Tecnología' 'Ciencias Naturales'
 'Ciencias Médicas y de la Salud' 'Ciencias Sociales' 'Ciencias Agrícolas'
 'Humanidades']
['C' 'A' 'B' 'A1' 'reconocido']


In [9]:
query_group_code = 'COL0035995' # sirius
group = df[df['code'] == query_group_code]
selected_knowledge_area = group['knowledgeArea'].values[0]
classification = group['classification'].values[0]
group_sumary = group.drop(['knowledgeArea', 'classification', 'code'], axis=1)

print('For group with code: {}'.format(query_group_code))
print('Knowledge area: {}'.format(selected_knowledge_area))
print('Classification: {}'.format(classification))

For group with code: COL0035995
Knowledge area: Ciencias Naturales
Classification: A1


In [10]:
group.head()

,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,TM,TG,PID,PF,PERS,AP,APO,knowledgeArea,classification,code
248,0.0,2.0,6.0,3.0,4.0,0.0,7.0,2.0,6.0,7.0,...,5.61,22.11,3.18,0.0,0.0,1.82,0.0,Ciencias Naturales,A1,COL0035995


In [25]:

KNOWLEDGE_AREAS = [
    'Ingeniería y Tecnología',
    'Ciencias Naturales',
    'Ciencias Médicas y de la Salud',
    'Ciencias Sociales',
    'Ciencias Agrícolas',
    'Humanidades',
]

CLASSIFICATIONS = ['A1', 'A', 'B', 'C', 'reconocido']
for area in KNOWLEDGE_AREAS:
    for classification in CLASSIFICATIONS:
        cluster = df[(df['knowledgeArea'] == area)
                     & (df['classification'] == classification) & (df['code'] != 'COL0035995')]
        print('Area: {}, Classification {}, Cluster Shaper {}'.format(area, classification, cluster.shape))

Area: Ingeniería y Tecnología, Classification A1, Cluster Shaper (129, 41)
Area: Ingeniería y Tecnología, Classification A, Cluster Shaper (158, 41)
Area: Ingeniería y Tecnología, Classification B, Cluster Shaper (205, 41)
Area: Ingeniería y Tecnología, Classification C, Cluster Shaper (403, 41)
Area: Ingeniería y Tecnología, Classification reconocido, Cluster Shaper (99, 41)
Area: Ciencias Naturales, Classification A1, Cluster Shaper (136, 41)
Area: Ciencias Naturales, Classification A, Cluster Shaper (153, 41)
Area: Ciencias Naturales, Classification B, Cluster Shaper (206, 41)
Area: Ciencias Naturales, Classification C, Cluster Shaper (372, 41)
Area: Ciencias Naturales, Classification reconocido, Cluster Shaper (102, 41)
Area: Ciencias Médicas y de la Salud, Classification A1, Cluster Shaper (91, 41)
Area: Ciencias Médicas y de la Salud, Classification A, Cluster Shaper (93, 41)
Area: Ciencias Médicas y de la Salud, Classification B, Cluster Shaper (233, 41)
Area: Ciencias Médicas y

## Let's compare it with groups of classification A

In [11]:
target_area = selected_knowledge_area
target_classification = classification
target_groups = df[(df['knowledgeArea'] == target_area) & (df['classification'] == classification) & (df['code'] != query_group_code)]
cluster_len = target_groups.shape[0]
print('Cluster size {}'.format(cluster_len))
target_groups.head()

Cluster size 136


,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,TM,TG,PID,PF,PERS,AP,APO,knowledgeArea,classification,code
577,0.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.91,3.47,0.00,0.0,0.00,2.69,0.0,Ciencias Naturales,A1,COL0116117
647,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.68,5.49,0.36,0.0,1.82,0.00,0.0,Ciencias Naturales,A1,COL0130949
671,0.0,1.0,6.0,4.0,2.0,1.0,1.0,0.0,1.0,4.0,...,5.77,3.94,6.20,0.0,0.00,0.00,0.0,Ciencias Naturales,A1,COL0004522
692,3.0,5.0,7.0,21.0,15.0,0.0,2.0,11.0,18.0,18.0,...,14.46,6.93,13.12,0.0,20.28,0.00,0.0,Ciencias Naturales,A1,COL0014823
695,1.0,3.0,4.0,3.0,3.0,0.0,0.0,4.0,2.0,0.0,...,10.54,3.94,1.46,0.0,0.00,0.00,0.0,Ciencias Naturales,A1,COL0060379


In [13]:
target_groups = df[(df['knowledgeArea'] == "Ciencias Veterinarias") & (df['classification'] == "C") & (df['code'] != query_group_code)]
cluster_len = target_groups.shape[0]
print('Cluster size {}'.format(cluster_len))
target_groups.head()

Cluster size 0


,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,TM,TG,PID,PF,PERS,AP,APO,knowledgeArea,classification,code


In [42]:
tree = spatial.KDTree(target_groups.drop(['knowledgeArea', 'classification', 'code'], axis=1))

In [55]:
closest = tree.query(group_sumary.values)
print(closest)
closest_group = target_groups.iloc[closest[1], :]
closest_group

(array([20.65453703]), array([123]))


,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,TM,TG,PID,PF,PERS,AP,APO,knowledgeArea,classification,code
4154,0.0,3.0,6.0,1.0,3.0,0.0,1.0,2.0,0.0,5.0,...,14.56,23.99,3.0,0.0,0.0,6.2,0.0,Ciencias Naturales,A1,COL0009135


In [58]:
group_sumary

,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,CCO,CCE,TD,TM,TG,PID,PF,PERS,AP,APO
248,0.0,2.0,6.0,3.0,4.0,0.0,7.0,2.0,6.0,7.0,...,1.68,7.88,0.0,5.61,22.11,3.18,0.0,0.0,1.82,0.0


In [59]:
closest_group.drop(['knowledgeArea', 'classification', 'code'], axis=1)

,IS,I,IJ,ED,EM,JI,EP,IVD,IVM,IVP,...,CCO,CCE,TD,TM,TG,PID,PF,PERS,AP,APO
4154,0.0,3.0,6.0,1.0,3.0,0.0,1.0,2.0,0.0,5.0,...,4.78,9.65,0.0,14.56,23.99,3.0,0.0,0.0,6.2,0.0


In [71]:
diff = closest_group.drop(['knowledgeArea', 'classification', 'code'], axis=1).values -  group_sumary.values
diff = np.abs(diff)
diff

array([[ 0.  ,  1.  ,  0.  ,  2.  ,  1.  ,  0.  ,  6.  ,  0.  ,  6.  ,
         2.  ,  3.  ,  5.  ,  0.09,  0.57,  8.47,  1.05,  1.07,  0.  ,
         0.  ,  0.  ,  0.  ,  3.36,  0.  ,  0.  ,  1.82,  0.  ,  0.  ,
        10.3 ,  3.1 ,  1.77,  0.  ,  8.95,  1.88,  0.18,  0.  ,  0.  ,
         4.38,  0.  ]])

In [72]:
named_list = list(zip(group_sumary.columns.tolist(), diff.squeeze().tolist()))
sorted_list = sorted(named_list, key=lambda tup: tup[1], reverse=True)
sorted_list

[('EPF', 10.3),
 ('TM', 8.95),
 ('ART_A', 8.469999999999999),
 ('EP', 6.0),
 ('IVM', 6.0),
 ('IV', 5.0),
 ('AP', 4.38),
 ('TEC', 3.36),
 ('CCO', 3.1000000000000005),
 ('IVE', 3.0),
 ('ED', 2.0),
 ('IVP', 2.0),
 ('TG', 1.879999999999999),
 ('CON', 1.82),
 ('CCE', 1.7700000000000005),
 ('LIB', 1.07),
 ('ART_D', 1.0499999999999998),
 ('I', 1.0),
 ('EM', 1.0),
 ('ICOOP', 0.5700000000000001),
 ('PID', 0.18000000000000016),
 ('IC', 0.09000000000000008),
 ('IS', 0.0),
 ('IJ', 0.0),
 ('JI', 0.0),
 ('IVD', 0.0),
 ('CAP', 0.0),
 ('PAT', 0.0),
 ('VV', 0.0),
 ('AAD', 0.0),
 ('EMP', 0.0),
 ('RNL', 0.0),
 ('MR', 0.0),
 ('PCI', 0.0),
 ('TD', 0.0),
 ('PF', 0.0),
 ('PERS', 0.0),
 ('APO', 0.0)]